In [2]:
import numpy as np
import pandas as pd
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
doc1 = "This is a very good and plain paper. this is really \
good and interesting"
doc2 = "This paper is very interesting, awesome"

In [4]:
def clean_txt(sent):
    tokens = word_tokenize(sent.lower())
    stop_updated = stopwords.words("english") + list(punctuation) 
    final_word = [term for term in tokens if term not in stop_updated 
               and len(term) > 2] 
    res = " ".join(final_word)
    return res

In [5]:
doc1_clean = clean_txt(doc1)
doc1_clean

'good plain paper really good interesting'

In [6]:
doc2_clean = clean_txt(doc2)
doc2_clean

'paper interesting awesome'

In [7]:
doc = pd.DataFrame([doc1_clean, doc2_clean], columns=["text"])
doc

,text
0,good plain paper really good interesting
1,paper interesting awesome


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
count_vect = CountVectorizer()

In [10]:
X = count_vect.fit_transform(doc['text'])
X

<2x6 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

fit -> extarct distinct words to form a bag of words
transform -> compute the occourrence of every word from the BOW in each document

In [11]:
#Obtaining the BOW's
count_vect.get_feature_names()

['awesome', 'good', 'interesting', 'paper', 'plain', 'really']

In [12]:
#Get the position of every word in the BOW, its usually dictionary ordered
print(count_vect.vocabulary_)

{'good': 1, 'plain': 4, 'paper': 3, 'really': 5, 'interesting': 2, 'awesome': 0}


In [13]:
#Printing the document term matrix can be done using toarray or todense()
#methods
X.toarray()

array([[0, 2, 1, 1, 1, 1],
       [1, 0, 1, 1, 0, 0]], dtype=int64)

In [14]:
X.todense()

matrix([[0, 2, 1, 1, 1, 1],
        [1, 0, 1, 1, 0, 0]], dtype=int64)

In [15]:
DTM = pd.DataFrame(X.toarray(),columns = count_vect.get_feature_names())
DTM

,awesome,good,interesting,paper,plain,really
0,0,2,1,1,1,1
1,1,0,1,1,0,0


In [16]:
TDM = DTM.T
TDM

,0,1
awesome,0,1
good,2,0
interesting,1,1
paper,1,1
plain,1,0
really,1,0


Creating Bigrams - bigrams are words taken in pairs. The next immediate word is grouped with the current word to form a bigram

1. unigram - one word
2. bigram - two words
3. trigram - three words
----- any combination of the above can be created
By default the vectorizer's in sklearn will perform on unigrams, i.e ngram_range will be set to (1,1)
to extarct bigrams ngram_range = (2,2)
to extract trigrams ngram_range = (3,3)
To extract both unigrams and bigrams ngram_range = (1,2)

In [17]:
# A token is called as a bigram if its min and max length is 2
count_vect_bg = CountVectorizer(ngram_range=(2,2))

In [18]:
X_bg = count_vect_bg.fit_transform(doc['text'])

In [19]:
print(count_vect_bg.get_feature_names())

['good interesting', 'good plain', 'interesting awesome', 'paper interesting', 'paper really', 'plain paper', 'really good']


In [20]:
DTM_bg = pd.DataFrame(X_bg.toarray(),
                      columns=count_vect_bg.get_feature_names())
DTM_bg

,good interesting,good plain,interesting awesome,paper interesting,paper really,plain paper,really good
0,1,1,0,0,1,1,1
1,0,0,1,1,0,0,0


In [21]:
count_vect_ubg = CountVectorizer(ngram_range=(1,2))

In [22]:
X_ubg = count_vect_ubg.fit_transform(doc['text'])

In [23]:
print(count_vect_ubg.get_feature_names())

['awesome', 'good', 'good interesting', 'good plain', 'interesting', 'interesting awesome', 'paper', 'paper interesting', 'paper really', 'plain', 'plain paper', 'really', 'really good']


In [24]:
DTM_ubg = pd.DataFrame(X_ubg.toarray(),
                      columns=count_vect_ubg.get_feature_names())
DTM_ubg

,awesome,good,good interesting,good plain,interesting,interesting awesome,paper,paper interesting,paper really,plain,plain paper,really,really good
0,0,2,1,1,1,0,1,0,1,1,1,1,1
1,1,0,0,0,1,1,1,1,0,0,0,0,0


In [25]:
# Get the DTM for all the trigrams - CountVectorizer(ngram_range=(3,3))

In [26]:
#Get only top n features in our DTM

In [27]:
#Get the top 3 features in our DTM based on their frequencies
#accross all the documents in the corpus by taking a column sum of all the words in the DTM
count_nfeatures = CountVectorizer(max_features=3)

In [28]:
X_nfeatures = count_nfeatures.fit_transform(doc['text'])

In [29]:
count_nfeatures.get_feature_names()

['good', 'interesting', 'paper']

In [30]:
DTM_nfeatures = pd.DataFrame(X_nfeatures.toarray(),
                            columns=count_nfeatures.get_feature_names())
DTM_nfeatures

,good,interesting,paper
0,2,1,1
1,0,1,1


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [88]:
tfidf_vect = TfidfVectorizer()

In [89]:
X = tfidf_vect.fit_transform(doc['text'])

In [90]:
tfidf = pd.DataFrame(X.toarray(), columns=tfidf_vect.get_feature_names())
tfidf

,awesome,good,interesting,paper,plain,really
0,0.000000,0.755256,0.268685,0.268685,0.377628,0.377628
1,0.704909,0.000000,0.501549,0.501549,0.000000,0.000000


In [84]:
?TfidfVectorizer